# Ансамблевые модели

## Задача классификации 

В этом практическом задании вы научитесь работать с ансамблевыми моделями. Мы начнем с задачи классификации итальянского вина на предмет его пренадлежности к одному из трех видов. Загрузите датасет `Wine Data Database` с помощью функции `load_wine` из модуля `sklearn.datasets`.

In [3]:
from sklearn.datasets import load_wine

X, y = load_wine(return_X_y=True)

Модель случайного леса для классификации представлена классом `RandomForestClassifier` из модуля `sklearn.ensemble`. Конструктор этого класса содержит аргумент `n_estimators`, который соответствует колличеству базовых алгоритмов в случайном лесе. Целью этого задания будет настройка этого параметра. Сравните модели случайных лесов с различным числом базовых алгоритмов `{1, 5, 10, 20}`. Что происходит с качеством случайного леса на тестовых данных при увеличении этого числа? Ответом на это задание `answer1` является лучшая оценка качества модели, округленная до трех знаков после запятой. Используйте `accuracy` как метрику качества и скользящий контроль `cross_val_score` как метод оценки качества модели. Установите параметр `cv = StratifiedKFold(4)`. Возьмите среднее значение оценки качества. Для каждой из моделей случайного леса используете `random_state=42` при создании нового экземпляра.

### *РЕШЕНИЕ*

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold


In [5]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3, stratify=y, random_state=42
)
accuracies = list()
for estimator in {1, 5, 10, 20}:
  random_forest = RandomForestClassifier(n_estimators=estimator, random_state=42)
  rf_model = random_forest.fit(x_train, y_train)
  predictions = rf_model.predict(x_test)
  result = round(accuracy_score(y_test, predictions),3)
  accuracies.append(result)
  print(f'Accuracy: {result}')
answer1 = max(accuracies)
print(f'Answer1: {answer1}')

Accuracy: 0.926
Accuracy: 0.963
Accuracy: 0.981
Accuracy: 0.963
Answer1: 0.981


In [6]:
cv = StratifiedKFold(4)
for estimator in {1, 5, 10, 20}:
  random_forest = RandomForestClassifier(n_estimators=estimator, random_state=42)
  rf_model = random_forest.fit(X, y)
  cv_score = cross_val_score(
    rf_model, X, y,
    scoring='accuracy', cv=cv
  )
  # print('Cross val score: {}'.format(cv_score))
  print('Mean cross val score: {:.4f}'.format(cv_score.mean()))

Mean cross val score: 0.8264
Mean cross val score: 0.9720
Mean cross val score: 0.9721
Mean cross val score: 0.9385


Далее сравните модель градиентного бустинга `GradientBoostingClassifier` из `sklearn.ensemble` с логистической регрессией `LogisticRegression` из `sklearn.linear_model` на этой выборке. Используете параметр `random_state=42` при создании экземпляров классов. Какая из моделей работает лучше? Приведите лучшую оценку, округленную до трех знаков после запятой, в качестве ответа `answer2` на это задание. Какие выводы из этого можно сделать?

### *РЕШЕНИЕ*

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

cv = StratifiedKFold(4)
grad_boosting = GradientBoostingClassifier(random_state=42)
gb_model = grad_boosting.fit(X, y)
cv_score = cross_val_score(
  gb_model, X, y,
  scoring='accuracy', cv=cv
)
print('Cross val score: {}'.format(cv_score))
print('Mean cross val score: {:.4f}'.format(cv_score.mean()))

log_regression = LogisticRegression(random_state=42, max_iter=3000)
lr_model = log_regression.fit(X, y)
cv_score = cross_val_score(
  gb_model, X, y,
  scoring='accuracy', cv=cv
)
print('Cross val score: {}'.format(cv_score))
print('Mean cross val score: {:.4f}'.format(cv_score.mean()))

Cross val score: [0.82222222 0.91111111 0.97727273 0.95454545]
Mean cross val score: 0.9163
Cross val score: [0.82222222 0.91111111 0.97727273 0.95454545]
Mean cross val score: 0.9163


## Задача регрессии

Загрузите уже известную вам выборку `Boston House Prices` и разделите ее случайным образом на тренировочную и тестовую выборку. Для этого используете функцию `train_test_split` с параметрами `random_state=54` и `test_size=0.33`. Мы будем сравнивать 4 модели: `RandomForestRegressor`, `GradientBoostingRegressor` из `sklearn.ensemble`, а так же Гребневую регрессию и ЛАССО (`Ridge`, `Lasso` из `sklearn.linear_model`). Обучите каждую модель на тренировочной выборке с параметром `random_state=42` в конструкторе. Какая из моделей показывает наименьшее значение среднеквадратической ошибки на тестовых данных? В качестве ответа `answer3` приведите это значение, округленное до двух цифр после запятой.

### *РЕШЕНИЕ*

In [11]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [14]:
boston = load_boston()

lasso = Lasso()
ridge = Ridge()
rforest = RandomForestRegressor()
gboosting = GradientBoostingRegressor()

for model in [lasso, ridge, rforest, gboosting]:
    x_train, x_test, y_train, y_test = train_test_split(
        boston.data, boston.target,
        test_size=0.33,
        random_state=54
    )
    model.fit(x_train, y_train)

    predictions = model.predict(x_test)
    print(model.__class__)
    print('MSE: {:.3f}\n'.format(mean_squared_error(y_test, predictions)))

/home/kraktorist/anaconda3/envs/coursera_ds/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the

<class 'sklearn.linear_model._coordinate_descent.Lasso'>
MSE: 26.917

<class 'sklearn.linear_model._ridge.Ridge'>
MSE: 23.796

<class 'sklearn.ensemble._forest.RandomForestRegressor'>
MSE: 8.923

<class 'sklearn.ensemble._gb.GradientBoostingRegressor'>
MSE: 8.517



# Строка с ответами

In [0]:
output = """Best score (random forest) {0:.3f}
Best score (other algorithms) {1:.3f}
Best score (regression) {2:.2f}"""
print(output.format(answer1, answer2, answer3))